In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.3.2`
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions._

val spark = SparkSession.builder()
  .appName("SparkTest")
  .master("local[*]")
  .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
  
// http://localhost:4040/jobs/
println("Spark is ready!")

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


Spark is ready!


import $ivy.$                                      

import org.apache.spark.sql._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions._


spark: SparkSession = org.apache.spark.sql.SparkSession@48764d69

In [2]:
import spark.implicits._

case class Customer(customer_id: Int, name: String, visited_on: java.sql.Date, amount: Int)

val customer: Seq[(Int, String, String, Int)] = Seq(
  (1, "Jhon", "2019-01-01", 100),
  (2, "Daniel", "2019-01-02", 110),
  (3, "Jade", "2019-01-03", 120),
  (4, "Khaled", "2019-01-04", 130),
  (5, "Winston", "2019-01-05", 110),
  (6, "Elvis", "2019-01-06", 140),
  (7, "Anna", "2019-01-07", 150),
  (8, "Maria", "2019-01-08", 80),
  (9, "Jaze", "2019-01-09", 110),
  (1, "Jhon", "2019-01-10", 130),
  (3, "Jade", "2019-01-10", 150)
)

val customerDS = customer.toDF("customer_id", "name", "visited_on", "amount")
  .withColumn("visited_on", $"visited_on".cast("date"))
  .as[Customer]

import spark.implicits._


defined class Customer
customer: Seq[(Int, String, String, Int)] = List(
  (1, "Jhon", "2019-01-01", 100),
  (2, "Daniel", "2019-01-02", 110),
  (3, "Jade", "2019-01-03", 120),
  (4, "Khaled", "2019-01-04", 130),
  (5, "Winston", "2019-01-05", 110),
  (6, "Elvis", "2019-01-06", 140),
  (7, "Anna", "2019-01-07", 150),
  (8, "Maria", "2019-01-08", 80),
  (9, "Jaze", "2019-01-09", 110),
  (1, "Jhon", "2019-01-10", 130),
  (3, "Jade", "2019-01-10", 150)
)
customerDS: Dataset[Customer] = [customer_id: int, name: string ... 2 more fields]

You are the restaurant owner and you want to analyze a possible expansion (there will be at least one customer every day).

Compute the moving average of how much the customer paid in a seven days window (i.e., current day + 6 days before). average_amount should be rounded to two decimal places.

Return the result table ordered by visited_on in ascending order.

The result format is in the following example.

| visited_on   | amount       | average_amount |
|--------------|--------------|----------------|
| 2019-01-07   | 860          | 122.86         |
| 2019-01-08   | 840          | 120            |
| 2019-01-09   | 840          | 120            |
| 2019-01-10   | 1000         | 142.86         |

In [3]:
val windowSpec = Window.rowsBetween(Window.currentRow - 6, Window.currentRow)
customerDS
    .groupBy(col("visited_on"))
    .agg(sum("amount").as("amount"))
    .withColumn("amount", sum(col("amount")).over(windowSpec))
    .withColumn("average_amount", round(col("amount") / lit(7), 2))
    .withColumn("day_cnt", sum(lit(1)).over(windowSpec))
    .filter(col("day_cnt") >= 7)
    .drop(col("day_cnt"))
    .sort(col("visited_on").asc)
    .show()

+----------+------+--------------+
|visited_on|amount|average_amount|
+----------+------+--------------+
|2019-01-07|   860|        122.86|
|2019-01-08|   840|         120.0|
|2019-01-09|   840|         120.0|
|2019-01-10|  1000|        142.86|
+----------+------+--------------+



windowSpec: WindowSpec = org.apache.spark.sql.expressions.WindowSpec@34f0cddb

In [4]:
spark.stop()